In [59]:
# Import Dependencies
import pandas as pd
import matplotlib as plt

In [70]:
# Import csv data

file_path = "Resources\Delays.csv"

delays_df = pd.read_csv(file_path, header=0)
delays_df.head()

,Date,FullShiftSuffix,DelayTimeMins,Eqmt,Reason,OperatorName
0,5/31/2016,Night Shift,0.033333,T380,LUBE CENTER,Orozco Matthew
1,6/1/2016,Day Shift,8.333333,T325,FIELD SERVICE,Lopez David
2,6/1/2016,Day Shift,11.100000,T317,FIELD SERVICE,FOSCA DAVID
3,6/1/2016,Day Shift,31.616666,T315,FIELD SERVICE,CLARK JOHN
4,6/1/2016,Day Shift,0.766666,T327,GENERAL OPERATIONS,Estep Frankie


In [71]:
# Check Data Types
delays_df.dtypes

Date                object
FullShiftSuffix     object
DelayTimeMins      float64
Eqmt                object
Reason              object
OperatorName        object
dtype: object

In [72]:
# Change Date to Date format
delays_df["Date"]= pd.to_datetime(delays_df["Date"])

delays_df.head()

,Date,FullShiftSuffix,DelayTimeMins,Eqmt,Reason,OperatorName
0,2016-05-31,Night Shift,0.033333,T380,LUBE CENTER,Orozco Matthew
1,2016-06-01,Day Shift,8.333333,T325,FIELD SERVICE,Lopez David
2,2016-06-01,Day Shift,11.100000,T317,FIELD SERVICE,FOSCA DAVID
3,2016-06-01,Day Shift,31.616666,T315,FIELD SERVICE,CLARK JOHN
4,2016-06-01,Day Shift,0.766666,T327,GENERAL OPERATIONS,Estep Frankie


In [73]:
# Check Equipment Category
delays_df["Eqmt"].unique()

array(['T380', 'T325', 'T317', 'T315', 'T327', 'T375', 'T326', 'T318',
       'T323', 'T320', 'T316', 'T321', 'T328', 'T322', 'S516', 'L530',
       'L531', 'T324', 'S517', 'S518', 'S520', 'T377', 'T319', 'T390',
       'T394', 'S515', 'L548', 'T391', 'T399', 'T395', 'T398', 'T329',
       'T393', 'L532'], dtype=object)

In [74]:
#  Drop Op name
delays_df = delays_df.drop(columns= "OperatorName", axis = 1)
delays_df.head()

,Date,FullShiftSuffix,DelayTimeMins,Eqmt,Reason
0,2016-05-31,Night Shift,0.033333,T380,LUBE CENTER
1,2016-06-01,Day Shift,8.333333,T325,FIELD SERVICE
2,2016-06-01,Day Shift,11.100000,T317,FIELD SERVICE
3,2016-06-01,Day Shift,31.616666,T315,FIELD SERVICE
4,2016-06-01,Day Shift,0.766666,T327,GENERAL OPERATIONS


In [75]:
# Create Truck_df and Loading_df

loading = ['S516', 'L530','L531','S517', 'S518', 'S520','S515', 'L548', 'L532']


Truck_df = delays_df[~delays_df["Eqmt"].isin(loading)]
loading_df = delays_df[delays_df["Eqmt"].isin(loading)]   

# Check success of removing loading
print(Truck_df["Eqmt"].unique())
print(loading_df["Eqmt"].unique())


['T380' 'T325' 'T317' 'T315' 'T327' 'T375' 'T326' 'T318' 'T323' 'T320'
 'T316' 'T321' 'T328' 'T322' 'T324' 'T377' 'T319' 'T390' 'T394' 'T391'
 'T399' 'T395' 'T398' 'T329' 'T393']
['S516' 'L530' 'L531' 'S517' 'S518' 'S520' 'S515' 'L548' 'L532']


In [76]:
# Drop Equipment Number
Truck_df.drop(columns="Eqmt", axis=1, inplace=True)


C:\Users\Joshua.Shutey\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [77]:
Truck_df

,Date,FullShiftSuffix,DelayTimeMins,Reason
0,2016-05-31,Night Shift,0.033333,LUBE CENTER
1,2016-06-01,Day Shift,8.333333,FIELD SERVICE
2,2016-06-01,Day Shift,11.100000,FIELD SERVICE
3,2016-06-01,Day Shift,31.616666,FIELD SERVICE
4,2016-06-01,Day Shift,0.766666,GENERAL OPERATIONS
...,...,...,...,...
288500,2022-05-03,Day Shift,5.133333,STEERING/BRAKE TEST
288501,2022-05-03,Day Shift,5.150000,STEERING/BRAKE TEST
288502,2022-05-03,Day Shift,5.933333,STEERING/BRAKE TEST
288503,2022-05-03,Day Shift,6.883333,STEERING/BRAKE TEST


In [81]:
# Sort Data with groupby to get total data/shift by day

truck_group_df = Truck_df.groupby(['Date','Reason']).sum()

truck_group_df.reset_index(inplace=True)
truck_group_df

,Date,Reason,DelayTimeMins
0,2016-05-31,LUBE CENTER,0.033333
1,2016-06-01,FIELD SERVICE,51.049999
2,2016-06-01,GENERAL OPERATIONS,12.533332
3,2016-06-01,LUBE CENTER,313.066662
4,2016-06-01,LUNCH,721.816656
...,...,...,...
21974,2022-05-03,FIELD SERVICE,73.866665
21975,2022-05-03,LUBE CENTER,105.516666
21976,2022-05-03,OPERATOR BREAK,79.016665
21977,2022-05-03,PRE-OP,211.683332
